# Processing Notebook

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Read csv

In [25]:
df: pd.DataFrame = pd.read_csv("neutron5cm_7cmSciOver.csv")
original_len = len(df['vx'])

In [29]:
nx,ny = 2,2
n = nx * ny

rep = {-1:0}
df['total'] = np.zeros_like(df['vx']).astype(int)
for i in range(n):
    col = f'photons{i}'
    df[col] = df[col].replace(rep)
    df['total'] += df[col]

print(sum(df['total'] > 0)/original_len)


0.6867777277502627


In [31]:
nx,ny = 2,2
n = nx * ny

rep = {-1:0}
sizes = [2,5,7]
for s in sizes:
    df = pd.read_csv(f"neutron5cm_{s}cmSciOver.csv")
    df['total'] = np.zeros_like(df['vx']).astype(int)
    original_len = len(df['vx'])
    for i in range(n):
        col = f'photons{i}'
        df[col] = df[col].replace(rep)
        df['total'] += df[col]

    print(sum(df['total'] > 0)/original_len)


0.7032774580935702
0.6870260299241648
0.6867777277502627


## Dynamically determine which columns are right or left and up or down

In [7]:
left_dropnames = [] # Columns to drop to get left subset
right_dropnames = [] # Columns to drop to get right subset
up_dropnames = [] # Columns to drop to get up subset
down_dropnames = [] # Columns to drop to get down subset
nx = 2
ny = 2
num = 0
for i in range(nx):
    for j in range(ny):
        if i < (nx / 2):
            print("left:",num)
            right_dropnames.append(f"photons{num}")
            right_dropnames.append(f"ypos{num}")
            right_dropnames.append(f"xpos{num}")
        else:
            print("right",num)
            left_dropnames.append(f"photons{num}")
            left_dropnames.append(f"ypos{num}")
            left_dropnames.append(f"xpos{num}")

        if j < (ny / 2):
            print("down:",num)
            up_dropnames.append(f"photons{num}")
            up_dropnames.append(f"ypos{num}")
            up_dropnames.append(f"xpos{num}")
        else:
            print('up:',num)
            down_dropnames.append(f"photons{num}")
            down_dropnames.append(f"ypos{num}")
            down_dropnames.append(f"xpos{num}")

        num += 1

left: 0
down: 0
left: 1
up: 1
right 2
down: 2
right 3
up: 3


## Create R,L,U,D Subsets and Find totals

In [4]:
left: pd.DataFrame = df.drop(left_dropnames,axis=1)
right: pd.DataFrame = df.drop(right_dropnames,axis=1)
up: pd.DataFrame = df.drop(up_dropnames,axis=1)
down: pd.DataFrame = df.drop(down_dropnames,axis=1)

In [ ]:
n = nx * ny

left['total'] = np.zeros_like(left['vx']).astype(int)
right["total"] = np.zeros_like(right['vx']).astype(int)
up['total'] = np.zeros_like(up['vx']).astype(int)
down['total'] = np.zeros_like(down['vx']).astype(int)
for i in range(n):
    # LEFT AND RIGHT
    try:
        left['total'] = left['total'] + left[f"photons{i}"]
    except:
        pass
    try:
        right['total'] = right['total'] + right[f"photons{i}"]
    except:
        pass

    # UP AND DOWN
    try:
        up['total'] = up['total'] + up[f'photons{i}']
    except:
        pass
    try:
        down['total'] = down['total'] + down[f'photons{i}']
    except:
        pass

df['total'] = left['total'] + right['total']

NOPHOTONS_LorR = np.logical_and(left['total']>0, right['total']>0) # Mask to drop all rows where no photons were detected on the left or the right side
NOPHOTONS_UorD = np.logical_and(up['total']>0, down['total']>0)
NOPHOTONS_All = np.logical_and(NOPHOTONS_LorR, NOPHOTONS_UorD)
NOTENOUGH = np.logical_and(df['total'] > 5, NOPHOTONS_All)

ONEEVENT = df['total'] == 1

In [ ]:
left = left[NOTENOUGH]
right = right[NOTENOUGH]
up = up[NOTENOUGH]
down = down[NOTENOUGH]
df = df[NOTENOUGH]

left = left.reset_index()
right = right.reset_index()
up = up.reset_index()
down = down.reset_index()
df = df.reset_index()

## Efficiency

In [ ]:
print(f'Efficiency: {len(df['vx']) / original_len * 100}%')

## Plots

In [ ]:
left_ratio = left['total']/df['total']
print(np.argmax(left_ratio))
sns.scatterplot(y=df["vx"],x=left_ratio)
plt.title("Left Ratio versus vx")
plt.xlabel("Left Ratio")
plt.ylabel("vx")

In [ ]:
right_ratio = right['total']/df['total']
sns.scatterplot(y=df["vx"],x=right_ratio)
plt.title("vx versus Right Ratio")
plt.xlabel("Right Ratio")
plt.ylabel("vx")

In [ ]:
up_ratio = up['total']/df['total']
sns.scatterplot(y=df["vy"],x=up_ratio)
plt.title("vy versus Up ratio")
plt.xlabel("Up Ratio")
plt.ylabel("vy")

In [ ]:
down_ratio = down['total']/df['total']
sns.scatterplot(y=df["vy"],x=down_ratio)
plt.title("vy versus Down Ratio")
plt.xlabel("Down Ratio")
plt.ylabel("vy")

## Curve Fits

In [ ]:
p = np.poly1d(np.polyfit(x=up_ratio,y=up['vy'],deg=3))
pred = p(up_ratio)
sns.scatterplot(x=up_ratio,y=up['vy'])
sns.lineplot(x=up_ratio,y=pred,c='black')
plt.xlabel('up ratio')
plt.ylabel('y vertex')
plt.title('y vertex versus ratio')

ybar = np.mean(up['vy'])
SST = sum((up['vy'] - ybar)**2)
SSReg = sum((pred - ybar)**2)
print(SSReg/SST)

In [ ]:
fig,ax = plt.subplots(1,1)
ax.set_facecolor('#ADD8E6')
ax.set_axisbelow(True)
ax.yaxis.grid(color='white', linestyle='-')
ax.xaxis.grid(color='white', linestyle='-')
sns.scatterplot(x=left_ratio,y=up['vx'])
for i in range(6):
    p = np.poly1d(np.polyfit(x=left_ratio,y=left['vx'],deg=i+1))
    pred = p(left_ratio)
    sns.lineplot(x=left_ratio,y=pred,label=f"deg = {i+1}")
plt.xlabel('left ratio')
plt.ylabel('x vertex')
plt.title('x vertex versus ratio (10 cm separation)')
plt.savefig("fits.jpg")

ybar = np.mean(left['vx'])
SST = sum((left['vx'] - ybar)**2)
SSReg = sum((pred - ybar)**2)
print(SSReg/SST)

## Standard Deviations

In [ ]:
degree = []
xsds = []
ysds = []
for i in range(10):
    p = np.poly1d(np.polyfit(x=left_ratio,y=left['vx'],deg=i))
    pred = p(left_ratio)
    xsds.append(np.std(pred-up['vx']))

    p = np.poly1d(np.polyfit(x=up_ratio,y=up['vy'],deg=i))
    pred = p(up_ratio)
    ysds.append(np.std(pred-up['vy']))

    degree.append(i)


fig,ax = plt.subplots(1,1)
sns.lineplot(ax=ax,x=degree,y=np.array(xsds)/10,label='x SD',c='salmon')
sns.lineplot(ax=ax,x=degree,y=np.array(ysds)/10,label='y SD',c='purple')
ax.set_facecolor('#ADD8E6')
ax.set_axisbelow(True)
ax.yaxis.grid(color='white', linestyle='-')
ax.xaxis.grid(color='white', linestyle='-')
plt.xlabel("Polyfit Degree")
plt.ylabel("Standard Deviation (cm)")
plt.title("Std vs degree (10 cm separation gate > 100)")
plt.savefig("degree.jpg",dpi=300)